<a href="https://colab.research.google.com/github/rimmedevries/2025-Projects-Testbed/blob/main/nsp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install <package>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install <package>'


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lmfit
from lmfit import models

ModuleNotFoundError: No module named 'lmfit'

# Data inladen met pandas, maak daar datalijsten van

In [ ]:
# Van data een lijst maken met 2 dimensies, sep=\t splits elke regel op een tab, decimal vervangt comma naar punt
df1 = pd.read_csv("LDA_eend_28cm_12microm.unknown", sep="\t", decimal=",", header=None, names=["Frequence","Intensity"])
frequencies = df1["Frequence"].tolist()
intensities = df1["Intensity"].tolist()

# Functie die nodig is voor fitten, maak daar een model van

In [ ]:
# Data loopt ongeveer als Gauss-functie, dus hieraan fitten
def GaussFunctie(x, A, x0, sigma) :
	return A * np.exp(-(x-x0)**2 / (2 * sigma**2))

# Maak model van functie
ons_model = models.Model(GaussFunctie)

# Fitten, let op juiste startwaardes anders krijg je niks
result = ons_model.fit(intensities, x=frequencies, A=0.5,x0=4500, sigma = 100)
print(result.fit_report())

# Bereken afwijking op frequentie met sigma uit fit report

In [ ]:
# Afwijking berekenen, voor frequentie dit met sigma doen van fitreport
sigma = result.params['sigma'].value
frequencies_err = (sigma*-1)/np.sqrt(len(frequencies))

intensities_err = np.std(intensities)/np.sqrt(len(frequencies))

# Plotten van fit

In [ ]:
plt.errorbar(frequencies, intensities, xerr=np.abs(frequencies_err), yerr=intensities_err, color= 'b',fmt='.', markersize=0.9)
plt.plot(frequencies, result.best_fit, color= 'b', alpha = 0.3)
#plt.plot(df["Frequence"], df["Intensity"], color= 'b', alpha = 0.3)
plt.title('Amplitude peak at frequency of particle passing by')
plt.xlabel("Frequence [Hz]")
plt.ylabel("Intensity")
# plt.gca().set_xlim([0, 6000])
plt.grid(True)
plt.show()

# Alle standaard meetwaardes



In [ ]:
# Definieer afwijkingen
y=0.101
r=0.085
D=.0025
_lambda = 632.8 *10**-9
dy=0.001
dr=0.0005
dD=0.0001

# Bereken alle snelheden van verschillende datafiles

In [ ]:
snelheden = []
sigma_f_list = []
snelheden_err =[]
afstanden_err = []
afstanden = []
afstanden_interferentiepunt = []

afstanden_metingen = [0,3,6,9,12,15]
for i in afstanden_metingen:
	df2 = pd.read_csv(f"LDA_eend_50cm_{i}microm.unknown", sep="\t", decimal=",", header=None, names=["Frequence","Intensity"])

	result2 = ons_model.fit(df2["Intensity"], x=df2["Frequence"], A=0.5,x0=4500, sigma = 100)
	f_max = result2.params['x0'].value #max. gefitte waarde van frequentie

	sigma_f = result2.params['sigma'].value # afwijking op max. frequentie
	sigma_f_list.append(sigma_f)

	# Bereken nieuwe totale afstand lens-interferentiepunt
	y_nieuw = y-((i-15)/1000)
	theta_lucht = np.arctan((r/2- D/2)/y_nieuw)

	# Wet van Snell
	theta_water = theta_lucht *0.75

	# Bereken afstand tussen laserbundels, hangt af van micrometerstand
	D = _lambda / (2*np.sin(theta_water))
	afstanden_interferentiepunt.append(D)

	L_lucht = D/(2 * np.tan(theta_lucht))
	L_water = D/(2 * np.tan(theta_water))

	# Afstand interferentiepunt tot as, volgens Wet van Snell
	totale_afstand = (L_water + L_lucht)
	afstanden.append(totale_afstand)

	# Afwijking op deze afstand laserbundels
	dD = np.sqrt((dy / y_nieuw)**2 + (dr / r)**2) *D
	# Foutenpropagatie van d, partiële afgeleiden: df/dy = r, df/dr = y, df/dD = 1
	sigma_d = float(np.sqrt((dy / (y-((i-15)/100)))**2 + (dr / r)**2 + (dD / D)**2)) *totale_afstand
	afstanden_err.append(sigma_d)

	# Snelheid bepalen
	v = f_max*totale_afstand
	snelheden.append(float(v))

	sigma_v = np.sqrt((sigma_f/f_max)**2 + (sigma_d/totale_afstand)**2) * v
	snelheden_err.append(sigma_v)


# Nieuwe fit van alle stroomsnelheden

In [ ]:
def parabool(x, a, b, c):
    return a*x**2 + b*x + c

parabool_model = models.Model(parabool)

result_snelheden = parabool_model.fit(snelheden, x=afstanden_metingen, a=0,b=0, c=0)
print(result_snelheden.fit_report())

# Plotten van v,R-diagram

In [ ]:
plt.errorbar(afstanden_metingen, snelheden, xerr=0.05, yerr= snelheden_err, color= 'b',fmt='.', markersize=0.9)
plt.plot(afstanden_metingen, result_snelheden.best_fit, color= 'b', alpha = 0.3)
plt.title('Velocity at different distances of tube')
plt.ylabel("Velocity [m/s]")
plt.xlabel("Distance in tube [mm]")
# plt.gca().set_xlim([0, 6000])
plt.grid(True)
plt.show()
